In [25]:
import os
import math
import random as rn

import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from keras import optimizers
from keras.preprocessing import image
from keras.applications import mobilenet_v2
from keras.utils.np_utils import to_categorical
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

In [26]:
SEED = 12345
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
rn.seed(SEED)

In [27]:
COLOR_MODE = "G"
EPOCHS = 100
BATCH_SIZE = 256

In [28]:
# Load MobileNet model
model = MobileNetV2(input_shape=(32,32,1), classes=10, include_top=True, weights=None)
opt = optimizers.Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
# Fetch the dataset directly
dataset = tfds.image_classification.Cifar10()

In [30]:
# Describe the dataset with DatasetInfo
C = dataset.info.features['label'].num_classes
Ntrain = dataset.info.splits['train'].num_examples
Ntest = dataset.info.splits['test'].num_examples
Nbatch = BATCH_SIZE
assert C == 10
assert Ntrain == 50000
assert Ntest == 10000

In [31]:
# Download the data, prepare it, and write it to disk
dataset.download_and_prepare()

In [32]:
# Load data from disk as tf.data.Datasets
datasets = dataset.as_dataset()
train_dataset, validation_dataset = datasets['train'], datasets['test']
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(validation_dataset, tf.data.Dataset)

In [33]:
def dataset_generator(dataset, batch_size=BATCH_SIZE, num_classes=10, is_training=False, color_mode="grayscale"):
  images = np.zeros((batch_size, 32, 32, 1))
  labels = np.zeros((batch_size, num_classes))
  while True:
    count = 0 
    for sample in tfds.as_numpy(dataset):
      image = sample["image"]
      label = sample["label"]

      if color_mode == "grayscale":
        # rgb to grayscale
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
      else: # keep only one channel in RGB
        channel_index = list('BGR').index(color_mode.upper())
        image = image[:,:,channel_index]

      images[count%batch_size] = mobilenet_v2.preprocess_input(np.expand_dims(cv.resize(image, (32, 32)), -1))
      labels[count%batch_size] = np.expand_dims(to_categorical(label, num_classes=num_classes), 0)
      
      count += 1
      if (count%batch_size == 0):
        yield images, labels

In [34]:
# Train on Cifar10
checkpoint_path = "Mobilenet/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.makedirs(checkpoint_dir, exist_ok=True)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor="val_loss",
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
    mode='min',
    period=10
)
    
csv_logger = keras.callbacks.CSVLogger('MobileNet_training.csv')

model.fit_generator(dataset_generator(train_dataset, batch_size=Nbatch, is_training=True, color_mode=COLOR_MODE),
                        steps_per_epoch= Ntrain // Nbatch,
                        epochs = EPOCHS,
                        validation_data = dataset_generator(validation_dataset, batch_size=Nbatch, color_mode=COLOR_MODE),
                        validation_steps = Ntest // Nbatch,
                        verbose = 1,
                        callbacks = [cp_callback, csv_logger])

Starting to train Modified MobileNet...
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


195/195 [==============================] - 43s 176ms/step - loss: 2.3018 - accuracy: 0.1752 - val_loss: 2.3029 - val_accuracy: 0.0999

Epoch 00001: val_loss improved from inf to 2.30291, saving model to Mobilenet/cp-0001.ckpt
Epoch 2/5
195/195 [==============================] - 25s 126ms/step - loss: 1.8116 - accuracy: 0.3313 - val_loss: 2.3049 - val_accuracy: 0.1002

Epoch 00002: val_loss did not improve from 2.30291
Epoch 3/5
 59/195 [========>.....................] - ETA: 14s - loss: 1.6496 - accuracy: 0.4060

KeyboardInterrupt: ignored

In [ ]:
# Infer on Cifar10
labels = np.zeros((Ntest))
pred_labels = np.zeros((Ntest, C))
pred_labels_new = np.zeros((Ntest, C))
  
score = model.evaluate_generator(
    dataset_generator(validation_dataset,batch_size=Nbatch),
    color_mode=COLOR_MODE,
    steps= Ntest // Nbatch,
    verbose=1,)

print("Evaluation Result of new Model on cifar100: " + str(score))

In [ ]:
model.save("MobileNetV2_{}.h5".format(COLOR_MODE))